## Total sea level occupation

#### Figure that shows the cumulative sea level occupation along bedrock elevation datum and inform potential of terrace creation preservation

Supplementary material for the manuscript submitted to Geology: Malatesta, L. C., Finnegan, N. J., Huppert, K., and Carreño, E.: The influence of rock uplift rate on the formation and preservation of individual marine terraces during multiple sea level stand,
Luca C. Malatesta, May 2021, luca.malatesta@gfz-potsdam.de, GitHub geo-luca

In [1]:
# switch to a graphics backend that allows interactions. standard is 'inline' which creates a static figure
# if plots don't show, try shutdown, and run 'jupyter lab build' in the terminal
%matplotlib ipympl

import numpy as np
import pandas as pd
import math
from   scipy import stats

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from   matplotlib.pyplot import figure
import mpl_interactions.ipyplot as iplt
from   matplotlib.widgets import Slider
import mpl_interactions.ipyplot as iplt
from   cmcrameri import cm # import scientific colormaps by Fabio Crameri...
                           # ... they can be installed with: conda install -c conda-forge matplotlib numpy pandas crameri

In [2]:
# parameters

Tmax    = 300      # maximum age of sea level record, ka
Uminmax = [0.0, 1.5] # range of rock uplift rate to test, mm/yr

In [15]:
# import sea level curves
# data should be organized in a csv file with age in ka in the first column and elevation in m in the second column.
# First row headers should be 'Age, ka' and 'Elevation, m'

# create dictionaries

dfs = {}        # dictionary for the dataframes
d_Tmax   = {}   # dictionary for maximum time
d_occup  = {}   # dictionary for 2D arrays to store occupation time
d_occupmax = {} # dictionary just for the max value of occupation
d_rcrd   = {}   # dictionary for 2D array storing relative SL
d_Zrange = {}   # dictionary for 1D range covering all elevations at max uplift

dfs['SL_SL16'] = pd.read_csv("./SLcurves/SL_Spratt_Lisiecki_16.csv") # import Spratt & Lisiecki, 2016
dfs['SL_Mi05'] = pd.read_csv("./SLcurves/SL_Miller_et_al_05.csv")    # import Miller et al., 2005, going back to 170Ma, too coarse for Pleistocene
dfs['SL_Le02'] = pd.read_csv("./SLcurves/SL_Lea_et_al_02.csv")       # import Lea et al., 2002
dfs['SL_LR05'] = pd.read_csv("./SLcurves/SL_Lisiecki_Raymo_05.csv")  # import Lisiecki & Raymo, 2005
dfs['SL_Wh20'] = pd.read_csv("./SLcurves/SL_westerhold_et_al_20.csv")# import Westerhold et al., 2020

d_source = {                                  # dictionary linking source key and full name
    'Lea et al., 2002':        'SL_Le02',
    'Lisiecki & Raymo, 2005':  'SL_LR05',
    'Miller et al., 2005':     'SL_Mi05',
    'Spratt & Lisiecki, 2016': 'SL_SL16',
    'Westerhold et al., 2020': 'SL_Wh20',
}
# don't forget to update the control command in the plot if you add additional SL curves

# references:
# D. W. Lea, P. A. Martin, D. K. Pak, and H. J. Spero. Reconstructing a 350ky history of sea level using planktonic mg/ca
# and oxygen isotope records from a cocos ridge core. Quaternary Science Reviews, 21(1):283–293, 2002. ISSN 0277-3791.
# doi: 10.1016/S0277-3791(01)00081-6.
# L. E. Lisiecki and M. E. Raymo. A pliocene-pleistocene stack of 57 globally distributed benthic δ18O records.
# Paleoceanography, 20(1), 2005. doi: 10.1029/2004PA001071.
# K.G. Miller, M.A. Kominz, J.V. Browning, J.D. Wright, G.S. Mountain, M.E. Katz, P.J. Sugarman, B.S. Cramer, N. Christie-Blick
# and S.F. Pekar, , 2005. The Phanerozoic record of global sea-level change. science, 310(5752), pp.1293-1298.
# R.M. Spratt, and L.E. Lisiecki, L.E., 2016, A Late Pleistocene sea level stack: Climate of the 324 Past, v. 12, p. 1079–1092,
# https://doi.org/10.5194/cp-12-1079-2016.
# T. Westerhold, N. Marwan, A.J. Drury, D. Liebrand, C. Agnini, E. Anagnostou, J.S. Barnet, S.M. Bohaty, D. De Vleeschouwer,
# F. Florindo and T. Frederichs, 2020. An astronomically dated record of Earth’s climate and its predictability over
# the last 66 million years. Science, 369(6509), pp.1383-1387.

In [16]:
# Choose and trim the sea level curve

for key in dfs.keys():   # returns combination of keys and values
    if Tmax > dfs[key]["Age, ka"].max():            # if Tmax is older than max age of record...
        d_Tmax[key] = dfs[key]["Age, ka"].max()   # ... adapt Tmax to actual max age.
    else:
        d_Tmax[key] = Tmax                        # otherwise keep Tmax defined at top
        
    dfs[key] = dfs[key][( dfs[key]["Age, ka"] <= d_Tmax[key] )]      # cut off SL older than max age

In [17]:
# Calculate distribution of sea level occupation under different uplift rates

U_range = np.linspace(Uminmax[0], Uminmax[1], 401)    # range of uplift rates
    
for key in dfs.keys():    # loop through dictionary keys
    Z_min   = np.floor( np.amin(dfs[key]["Elevation, m"].values + dfs[key]["Age, ka"].values * Uminmax[0]) ) # lowest elevation in array
    Z_max   = np.ceil ( np.amax(dfs[key]["Elevation, m"].values + dfs[key]["Age, ka"].values * Uminmax[1]) ) # highest elevation in array
    d_Zrange[key] = np.linspace(Z_min, Z_max, 401)   # range of elevations
    
    d_occup[key] = np.zeros( (d_Zrange[key].size, U_range.size) )   # array to store SL occupation as a function of elev. and uplft rate
    d_rcrd[key]  = np.zeros( (U_range.size, len(dfs[key])))           # array to store RSL as a function of uplift rate
    
    for i in range(0, U_range.size):     # start loop 
        RSL = dfs[key]["Elevation, m"].values + dfs[key]["Age, ka"].values * U_range[i]    # relative sea level under current uplift
        d_rcrd[key][i,:] = RSL                                                           # store RSL in the dictionary

        RSL_kernel = stats.gaussian_kde(RSL)                              # compute kernel distribution estimate
        RSL_kernel.set_bandwidth(bw_method=RSL_kernel.factor / 5.)        # modify the bandwith
        d_occup[key][:,i] = RSL_kernel(d_Zrange[key]) * d_Tmax[key] # multiply by total time and save distribution in array as kyr
    d_occupmax[key] = np.amax(d_occup[key])

In [18]:
# Plotting

# extract some values for axes limits:
ymin = Tmax * np.amin(U_range) -150
ymax = Tmax * np.amax(U_range)
max_occup = max(d_occupmax.values())  # max occup time of all RSLs (x-axis limit in center plot)

# setup the plotting frame xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
fig1 = plt.figure(constrained_layout=True)
fig1.set_size_inches(9, 3)
spec1 = gridspec.GridSpec(ncols=7, nrows=1, figure=fig1) 
f1_ax1 = fig1.add_subplot(spec1[0, 0:3])
f1_ax2 = fig1.add_subplot(spec1[0, 3])
f1_ax3 = fig1.add_subplot(spec1[0, 4:])

Uplift_rate = U_range      # parameter for plot interactivity

# define functions for the interactive plot xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
def f1x(Uplift_rate, type_):              # age of record, x axis
    return dfs[d_source[type_]]["Age, ka"].values

def f1y(f1x, Uplift_rate, type_):         # relative sea level, y axis
    return d_rcrd[d_source[type_]][np.argwhere(U_range == Uplift_rate).squeeze()]

def f1ybis(f1x, Uplift_rate, type_):      # reference sea level, y axis
    return dfs[d_source[type_]]["Elevation, m"]

# define functions for total sea level occupation xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
def f2x(Uplift_rate, type_):              # total occupation time, x axis
    return d_occup[d_source[type_]][:,np.argwhere(U_range == Uplift_rate).squeeze()]
    
def f2y(f2x, Uplift_rate, type_):         # elevation range, y axis
    return d_Zrange[d_source[type_]]

# define function for total sea level occupation heatmap xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
def f3M (Uplift_rate, type_):             # full map of occupation times (2D array)
    return d_occup[d_source[type_]]
    
def f3x(Uplift_rate, type_):              # uplift rate, x axis
    return Uplift_rate
    

# LEFT PLOT, relative sea level  xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
controls = iplt.plot(f1x, f1y, Uplift_rate=Uplift_rate,
                     type_={("Lea et al., 2002","Lisiecki & Raymo, 2005","Miller et al., 2005","Spratt & Lisiecki, 2016",
                             "Westerhold et al., 2020")},
                     ax=f1_ax1, color='darkblue')  # indicate, x, function, variable
iplt.plot(f1x, f1ybis, controls=controls, color='darkblue', linewidth=0.5, ax=f1_ax1)
f1_ax1.hlines(0, 0, Tmax, color='dimgrey',linewidth=1)         # plot a horizontal line at modern sea level
f1_ax1.set_ylim(ymin, ymax)

f1_ax1.grid(which='major', axis='y', color='lightgrey', linewidth=0.5)    # add horizontal grid lines
f1_ax1.set_ylim(ymin, ymax)
f1_ax1.set_ylabel('Elevation above modern sea level, m')
f1_ax1.set_xlabel('Age, ka')  
f1_ax1.set_title('Relative sea level')

# CENTER PLOT, cumulative occupation xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
iplt.plot(f2x, f2y, controls=controls,
          ax=f1_ax2, color='mediumslateblue',linewidth=2)
f1_ax2.hlines(0, 0, max_occup, color='dimgrey',linewidth=1)    # plot a horizontal line at modern sea level

f1_ax2.grid(which='major', axis='y', color='lightgrey', linewidth=0.5)            # add horizontal grid lines
f1_ax2.set_ylim(ymin, ymax)
f1_ax2.set_xlabel('Occupation, kyr')  

# RIGHT PLOT, heatmap of sea level occupation xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
iplt.imshow(f3M, origin='lower', interpolation='bilinear', cmap=cm.devon_r, extent=[Uminmax[0],Uminmax[1],Z_min,Z_max],
            aspect='auto', controls=controls, ax=f1_ax3)
clb = plt.colorbar(shrink = 0.4)
clb.set_label('total occupation, kyr')

f1_ax3.hlines(0, Uminmax[0], Uminmax[1], color='dimgrey',linewidth=1)         # plot a horizontal line at modern sea level
iplt.axvline(f3x, ymin, ymax, controls=controls, ax=f1_ax3, color='silver',linewidth=3) # plot vertical line at current sampled uplift

# cbar = fig.colorbar(cs, cax=cax)      # still to be figured out.
f1_ax3.grid(which='major', axis='y', color='lightgrey', linewidth=0.5)    # add horizontal grid lines
f1_ax3.set_ylim(ymin, ymax)
f1_ax3.set_ylabel('')  
f1_ax3.set_xlabel('Rock uplift rate, mm/yr')
f1_ax3.set_title('Total sea level occupation');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …